In [ ]:
# !unzip train_images.zip -d data

In [ ]:
# !unzip train_masks.zip -d data

In [ ]:
# !unzip val_images.zip -d data

In [ ]:
# !unzip val_masks.zip -d data

In [ ]:
# !rm -rf data/train_images

In [ ]:
# !rm -rf data/train_masks

In [ ]:
# !ls

In [ ]:
!python train.py

=> Loading checkpoint
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Got 3653761/3686400 with acc 99.11
Dice score: 0.9790122509002686


In [ ]:
# !pip install albumentations

In [ ]:
# !pip uninstall albumentations

In [ ]:
# !pip install albumentations==0.4.6

In [ ]:
# mkdir adv_dir
# mkdir target_adv_maskdir
# mkdir saved_images
# mkdir adv_maskdir

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
from model import UNET
from utils import (
    load_checkpoint,
    save_checkpoint,
    get_loaders,
    check_accuracy,
    save_predictions_as_imgs,
)

In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np
from dataset import CarvanaDataset
from torch.utils.data import DataLoader

In [ ]:
import albumentations as A

In [ ]:
from albumentations.pytorch import ToTensorV2

In [ ]:
from model import DoubleConv

In [ ]:
from dataset import CarvanaDataset

In [ ]:
# Hyperparameters etc.
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 1
NUM_EPOCHS = 0
NUM_WORKERS = 2
IMAGE_HEIGHT = 160  # 1280 originally
IMAGE_WIDTH = 240  # 1918 originally
PIN_MEMORY = True
LOAD_MODEL = False
TRAIN_IMG_DIR = "data/train_images/"
TRAIN_MASK_DIR = "data/train_masks/"
VAL_IMG_DIR = "data/val_images/"
VAL_MASK_DIR = "data/val_masks/"

In [ ]:
pretrained_model = "my_checkpoint.pth.tar"
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
device

device(type='cuda')

In [ ]:
model = UNET().to(device)
# model.load_state_dict(torch.load(pretrained_model))
load_checkpoint(torch.load("my_checkpoint.pth.tar"), model)
model.eval()

=> Loading checkpoint


UNET(
  (ups): ModuleList(
    (0): ConvTranspose2d(1024, 512, kernel_size=(2, 2), stride=(2, 2))
    (1): DoubleConv(
      (conv): Sequential(
        (0): Conv2d(1024, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (2): ConvTranspose2d(512, 256, kernel_size=(2, 2), stride=(2, 2))
    (3): DoubleConv(
      (conv): Sequential(
        (0): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), p

In [ ]:
train_transform = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Rotate(limit=35, p=1.0),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

In [ ]:
val_transforms = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

In [ ]:
train_loader, val_loader = get_loaders(
        TRAIN_IMG_DIR,
        TRAIN_MASK_DIR,
        VAL_IMG_DIR,
        VAL_MASK_DIR,
        BATCH_SIZE,
        train_transform,
        val_transforms,
        NUM_WORKERS,
        PIN_MEMORY,
    )

In [ ]:
for batch_idx, (data, targets) in enumerate(train_loader):
    print(batch_idx)
    print(data.size())
    break

0
torch.Size([1, 3, 160, 240])


In [ ]:
import string
import random
import time

In [ ]:
def test(model, device, dl, epsilon):
  adversarial_images = []
  correct = 0
  loss_fn = nn.BCEWithLogitsLoss()
  name = string.ascii_lowercase

  for batch_idx, (data, targets) in enumerate(dl):
    name = ''.join(random.choice(name) for i in range(10))

    data = data.to(device)

    targets = targets.float().unsqueeze(1).to(device=DEVICE)
    
    data.requires_grad = True

    output = model(data)
    
    for i in range(5):
      
      loss = loss_fn(output, targets)

      model.zero_grad()

      loss.backward()

      data_grad = data.grad.data.sign()

      perturbed_image = data + epsilon*data_grad

      data = perturbed_image.clone().detach()

      data.requires_grad_()

      output = model(data)

      adv_mask = model(perturbed_image)

    #Saving Adv Image and Corresponding Target Mask for Accuracy
    torchvision.utils.save_image(data, f"adv_dir/{batch_idx}{name}.jpg")
    time.sleep(.001)
    torchvision.utils.save_image(targets, f"target_adv_maskdir/{batch_idx}{name}_mask.gif")
    time.sleep(.001)


    # Saving Adv Masks for Fun
    torchvision.utils.save_image(adv_mask, f"adv_maskdir/{batch_idx}{name}_advmask.jpg")
    time.sleep(.001)

    # if batch_idx == 3:
    #   break


In [ ]:
epsilon = .09
# for i in range(10):
test(model, device, val_loader, epsilon)

In [ ]:
print(len(val_loader))

96


In [ ]:
ADV_DIR = "adv_dir/"
ADV_MASK_DIR = "adv_maskdir/"
TARGET_ADV_MASKDIR = "target_adv_maskdir/"

In [ ]:
def get_adv_loader(
    adv_dir,
    adv_maskdir,
    batch_size,
    train_transform,
    num_workers=4,
    pin_memory=True,
):
    adv_ds = CarvanaDataset(
        image_dir=adv_dir,
        mask_dir=adv_maskdir,
        transform=train_transform,
    )

    adv_loader = DataLoader(
        adv_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False,
    )

    return adv_loader

In [ ]:
# !rm adv_dir/.ipynb_checkpoints
adv_loader = get_adv_loader(
        ADV_DIR, 
        TARGET_ADV_MASKDIR, 
        BATCH_SIZE,
        val_transforms,
        NUM_WORKERS,
        PIN_MEMORY, )
# adv_loader

#CHECK ACCURACRY HERE
check_accuracy(adv_loader, model, device=DEVICE)

Got 2812665/3686400 with acc 76.30
Dice score: 0.2321237027645111


In [ ]:
# rmdir adv_dir/.ipynb_checkpoints

In [ ]:
!rm -rf adv_dir
!rm -rf target_adv_maskdir
!rm -rf adv_maskdir

!mkdir adv_dir
!mkdir target_adv_maskdir
!mkdir adv_maskdir